In [15]:
import jieba
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import WikiExtractor

In [16]:
# 读取停用词表
stop_lst = []
with open("stopwords.txt", encoding = 'UTF-8') as f:
    for line in f:
        stop_lst.append(line.strip())
f.close()

In [17]:
# 数据清洗
def cut2(x):
    segs = []
    for word in jieba.cut(x):
        if word not in stop_lst:
            segs.append(word)
    return segs

In [18]:
# 切分
def cut(x):
    return ' '.join(jieba.cut(x))

In [13]:
cut('无额外依赖，安装和使用都非常方便')

'无 额外 依赖 ， 安装 和 使用 都 非常 方便'

In [28]:
news = pd.read_csv('sqlResult_1558435.csv', encoding = 'gb18030')        # 读取语料
news = pd.DataFrame(news['content'].astype(str))        # 转类型，防止训练时报错
news['seg'] = news['content'].apply(cut)        # 切分
news['seg'].to_csv('articles_after_segs.txt', sep='\t', index=False)    # 保存切分后的文本
print(news['seg'].shape)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


(89611,)


In [29]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, word2vec

In [30]:
path = get_tmpfile("word2vec.model")        # 创建临时文件
sentences = word2vec.LineSentence('articles_after_segs.txt')        # 读取预料
model = Word2Vec(sentences, size=100, window=5, min_count=3, workers=4)        # 训练模型
model.save("word2vec.model")         # 保存模型
model.wv.save_word2vec_format('word2vec_format.txt')        # 将词向量以文本形式输出

In [33]:
# 测试线性相关性
def analogy(x1, x2, y1):
    results = model.most_similar(positive = [y1, x2], negative = [x1])
    return results[0][0]

In [ ]:
# model  = Word2Vec.load("word2vec.model")        # 加载已生成的模型

In [35]:
# 测试训练结果
print(model['西安'])
print(analogy("中国","汉语","美国"))
print(analogy("美国","奥巴马","美国"))
print(model.most_similar("西安"))
print(model.similarity("西安","陕西"), model.similarity("陕西","西安"))

[ 1.2978063   2.6064162  -2.332619   -0.57071877 -0.320048   -2.0576591
 -1.3851141  -0.42319807 -1.9019232   1.4114864   1.3061066   0.23547393
 -1.6191466  -0.5653941  -2.2620535   1.9745442   0.59777    -1.2853051
  0.8380661  -1.7132071   1.3818291  -1.4071718  -0.29351586  1.495115
 -0.28616434 -3.0190885   0.24029952 -1.3321489   0.8075312  -0.9078932
 -1.2885839   1.0601299   2.3225062  -0.9289253  -0.43842074 -2.9654613
 -0.09909485  0.6485282  -0.28908855  0.07933731 -0.10842301 -0.2648251
 -0.78850645 -1.4163364  -0.43824238 -1.8562491  -2.1682348   0.24986763
 -1.0223593  -0.33955255  1.149272   -1.3910553   1.7697997  -0.6296932
 -0.37255365 -0.72669876  3.9547708  -1.1725627  -0.05293956 -0.07772388
 -1.8258156  -0.32077938  0.42504543  0.01664756  1.5579945   0.6566446
  0.56389356 -1.6125147  -3.012735   -2.4776838   0.5371164   0.6330763
 -1.3960207   1.782439   -1.408616   -0.17154063 -2.385721   -0.7745727
 -1.3099456   0.472981   -0.7413956  -1.898522   -0.4192785   

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
